In [1]:
# Librerias que utiliza este aplicativo.
from flask import Flask, render_template, request, session, make_response, escape, url_for, redirect, flash
from flask_debug import Debug
import mysql.connector as mariadb
import hashlib

app = Flask(__name__)
app.secret_key = '123456'

# Conexión a la base de datos.
def BaseDatos():
    return mariadb.connect(host="localhost", port="3306", user="root", password="123456", database="python")

# Consulta para el área de contacto.
def InformacionContacto():
    Contacto = []
    
    conexion = BaseDatos()
    cursor = conexion.cursor()
    try:
        cursor.execute("SELECT correo, telefono, celular, direccion FROM contacto")
        for correo, telefono, celular, direccion in cursor:
            Contacto = [correo, telefono, celular, direccion]
    except mariadb.Error as error:
        print("Error: {}".format(error))
        
    conexion.close()
    return Contacto

def InformacionQS():
    Informacion = ''
    
    conexion = BaseDatos()
    cursor = conexion.cursor()
    try:
        cursor.execute("SELECT info FROM informacion")
        for info in cursor:
            Informacion = info
    except mariadb.Error as error:
        print("Error: {}".format(error))
        
    conexion.close()
    return Informacion

def ValidarUsuario(usuario, clave):
    conexion = BaseDatos()
    cursor = conexion.cursor()
    try:
        cursor.execute("""SELECT count(*) FROM usuarios WHERE documento = %s AND contrasena = %s""", (usuario, clave))
        resultado = cursor.fetchone()
        if(resultado[0] > 0):
            mensaje = "exito"
        else:
            mensaje = "error"
    except mariadb.Error as error:
        print("Error: {}".format(error))
    
    conexion.close()
    return mensaje

def ValidarContacto(correo, telefono, celular, direccion):
    conexion = BaseDatos()
    cursor = conexion.cursor()
    try:
        cursor.execute("""UPDATE contacto SET correo = %s, telefono = %s, celular = %s, direccion = %s""", (correo, telefono, celular, direccion))
        conexion.commit()
        if(cursor.rowcount > 0):
            mensaje = "exito"
        else:
            mensaje = "error"
    except mariadb.Error as error:
        print("Error: {}".format(error))
    
    conexion.close()
    return mensaje

def ValidarInformacion(info):
    conexion = BaseDatos()
    cursor = conexion.cursor()
    try:
        cursor.execute("""UPDATE informacion SET info = %s""", (info,))
        conexion.commit()
        if(cursor.rowcount > 0):
            mensaje = "exito"
        else:
            mensaje = "error"
    except mariadb.Error as error:
        print("Error: {}".format(error))
    
    conexion.close()
    return mensaje

@app.route("/")
def index():
    return render_template("index.php")

@app.route("/contacto", methods=["GET", "POST"])
def contacto():
    if request.method == "POST":
        correo = request.form["correo"]
        telefono = request.form["telefono"]
        celular = request.form["celular"]
        direccion = request.form["direccion"]
        if(ValidarContacto(correo, telefono, celular, direccion) == "exito"):
            flash("Información de contacto modificada correctamente.", "success")
        else:
            flash("Lo siento, hubo un error al momento de modificar la información de contacto.", "error")
    Contacto = []
    Contacto = InformacionContacto()
    return render_template("contacto.php", informacion=Contacto)

@app.route("/informacion", methods=["GET", "POST"])
def informacion():
    if request.method == "POST":
        info = request.form["info"]
        if(ValidarInformacion(info) == "exito"):
            flash("Información de quiénes sómos ha sido modificada correctamente.", "success")
        else:
            flash("Lo siento, hubo un error al momento de modificar la información de quiénes sómos.", "error")
    return render_template("informacion.php", informacion=InformacionQS())

@app.route("/productos", methods=["GET", "POST"])
def productos():
    return render_template("productos.php")

@app.route("/panel", methods=["GET", "POST"])
def panel():
    if request.method == "POST":
        usuario = request.form["usuario"]
        clave = hashlib.md5(str(request.form["clave"]).encode("utf-8")).hexdigest()
        if(ValidarUsuario(usuario, clave) == "exito"):
            session['usuario'] = usuario
            flash("Bienvenido, ha iniciado sesión correctamente.", "success")
            return redirect(url_for('index'))
        else:
            flash("Lo siento, los datos ingresados no son válidos.", "error")
    return render_template("panel.php")

@app.route("/logout")
def cerrar():
    del session['usuario']
    flash("Ha cerrado sesión correctamente, ¡Nos vemos más tarde!", "success")
    return render_template("index.php")

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Oct/2019 15:43:59] "GET /productos HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:44:43] "GET /productos HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:44:44] "GET /static/images/index/favicon.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:44:52] "GET /productos HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:44:52] "GET /static/images/index/favicon.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:47:02] "GET /productos HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:47:02] "GET /static/images/productos/1.png HTTP/1.1" 404 -
127.0.0.1 - - [10/Oct/2019 15:47:02] "GET /static/images/index/favicon.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:47:22] "GET /productos HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:47:22] "GET /static/images/productos/1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:47:22] "GET /static/images/index/favicon.png HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2019 15:47:45] "GET /productos 